This file reads data from NYC Open Data using the Socrata Open Data API.
Reads relevant files for the project.

In [1]:
import pandas as pd
from sodapy import Socrata
import sys

In [2]:
from IPython.display import display
def all_cols(df):
    with pd.option_context('display.max_columns', None):
        return display(df)

Sample pull

In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(
    "usc3-8zwd", content_type='json', limit = 2)

# Convert to pandas DataFrame
df_sample = pd.DataFrame(results)
df_sample.columns.tolist()

['property_id',
 'property_name',
 'parent_property_id',
 'parent_property_name',
 'year_ending',
 'nyc_borough_block_and_lot_bbl',
 'nyc_building_identification_number_bin',
 'address_1',
 'city',
 'postcode',
 'primary_property_type_self_selected',
 'primary_property_type_portfolio_manager_calculated',
 'national_median_reference_property_type',
 'list_of_all_property_use_types_at_property',
 'largest_property_use_type',
 'largest_property_use_type_gross_floor_area_ft',
 '_2nd_largest_property_use_type',
 '_2nd_largest_property_use_gross_floor_area_ft',
 '_3rd_largest_property_use_type',
 '_3rd_largest_property_use_type_gross_floor_area_ft',
 'year_built',
 'construction_status',
 'number_of_buildings',
 'occupancy',
 'metered_areas_energy',
 'metered_areas_water',
 'energy_star_score',
 'national_median_energy_star_score',
 'target_energy_star_score',
 'reason_s_for_no_score',
 'energy_star_certification_year_s_certified_score',
 'energy_star_certification_eligibility',
 'energy_sta

### Pull data

In [4]:
# Initialize Socrata client
client = Socrata("data.cityofnewyork.us", None)

#### 2020

Columns to pull

In [5]:
selected_columns_2020 = [
    'property_id', 'nyc_building_identification_number_bin', 
    'property_name', 'year_ending', 'city', 'postcode', 
    'list_of_all_property_use_types_at_property', 'largest_property_use_type', 
    '_2nd_largest_property_use_type', '_3rd_largest_property_use_type', 
    'largest_property_use_type_gross_floor_area_ft', 
    '_2nd_largest_property_use_gross_floor_area_ft', 
    '_3rd_largest_property_use_type_gross_floor_area_ft', 'year_built', 
    'occupancy', 'number_of_buildings', 'energy_star_score', 
    'source_eui_kbtu_ft', 'weather_normalized_source_eui_kbtu_ft', 
    'source_energy_use_kbtu', 'weather_normalized_source_energy_use_kbtu', 
    'weather_normalized_site_electricity_intensity_kwh_ft', 
    'natural_gas_use_kbtu', 'weather_normalized_site_natural_gas_use_therms', 
    'electricity_use_grid_purchase_kbtu', 
    'electricity_use_generated_from_onsite_renewable_systems_kwh', 
    'total_ghg_emissions_metric_tons_co2e', 
    'total_ghg_emissions_intensity_kgco2e_ft', 
    'net_emissions_metric_tons_co2e', 
    'national_median_total_ghg_emissions_metric_tons_co2e', 
    'estimated_data_flag_electricity_grid_purchase', 
    'estimated_data_flag_natural_gas', 'estimated_values_energy', 
    'default_values', 'temporary_values', 'property_gfa_self_reported_ft', 
    'property_gfa_calculated_parking_ft', 'borough', 'community_board', 
    'council_district', 'census_tract', 'nta', 'longitude', 'latitude'
]
mfh_columns = [c for c in df_sample.columns if c.startswith('multifamily_housing_')]
columns = ', '.join(selected_columns_2020 + mfh_columns)
# columns

Filter to existing, metered, multifamily housing.

In [6]:
mfh1 = 'primary_property_type_self_selected = "Multifamily Housing"'
mfh2 = 'primary_property_type_portfolio_manager_calculated = "Multifamily Housing"'
existing = 'construction_status = "Existing"'
metered = 'metered_areas_energy = "Whole Property"'
e_star = 'NOT energy_star_score = "Not Available"'
where = ' AND '.join([mfh1, mfh2, existing, metered, e_star])
where

'primary_property_type_self_selected = "Multifamily Housing" AND primary_property_type_portfolio_manager_calculated = "Multifamily Housing" AND construction_status = "Existing" AND metered_areas_energy = "Whole Property" AND NOT energy_star_score = "Not Available"'

In [7]:
# get all data from 2020 data set
dataset_id = "usc3-8zwd"
results = client.get_all(
    dataset_id, content_type='json', 
    select = columns,
    where = where)

# Convert to pandas DataFrame
df2020 = pd.DataFrame(results)
df2020.shape

(16894, 59)

In [8]:
all_cols(df2020.head())

,property_id,nyc_building_identification_number_bin,property_name,year_ending,city,postcode,list_of_all_property_use_types_at_property,largest_property_use_type,_2nd_largest_property_use_type,_3rd_largest_property_use_type,largest_property_use_type_gross_floor_area_ft,_2nd_largest_property_use_gross_floor_area_ft,_3rd_largest_property_use_type_gross_floor_area_ft,year_built,occupancy,number_of_buildings,energy_star_score,source_eui_kbtu_ft,weather_normalized_source_eui_kbtu_ft,source_energy_use_kbtu,weather_normalized_source_energy_use_kbtu,weather_normalized_site_electricity_intensity_kwh_ft,natural_gas_use_kbtu,weather_normalized_site_natural_gas_use_therms,electricity_use_grid_purchase_kbtu,electricity_use_generated_from_onsite_renewable_systems_kwh,total_ghg_emissions_metric_tons_co2e,total_ghg_emissions_intensity_kgco2e_ft,net_emissions_metric_tons_co2e,national_median_total_ghg_emissions_metric_tons_co2e,estimated_data_flag_electricity_grid_purchase,estimated_data_flag_natural_gas,estimated_values_energy,default_values,temporary_values,property_gfa_self_reported_ft,property_gfa_calculated_parking_ft,borough,community_board,council_district,census_tract,nta,longitude,latitude,multifamily_housing_government_subsidized_housing,multifamily_housing_gross_floor_area_ft,multifamily_housing_number_of_bedrooms,multifamily_housing_number_of_bedrooms_density_number_per_1_000_sq_ft,multifamily_housing_number_of_laundry_hookups_in_all_units,multifamily_housing_number_of_laundry_hookups_in_common_area_s,multifamily_housing_number_of_residential_living_units_in_a_high_rise_building_10_or_more_stories,multifamily_housing_total_number_of_residential_living_units,multifamily_housing_total_number_of_residential_living_units_density_number_per_1_000_sq_ft,multifamily_housing_number_of_residential_living_units_in_a_high_rise_building_density_number_per_1_000_sq_ft,multifamily_housing_number_of_residential_living_units_in_a_mid_rise_building_5_9_stories,multifamily_housing_number_of_residential_living_units_in_a_mid_rise_building_density_number_per_1_000_sq_ft,multifamily_housing_percent_that_can_be_cooled,multifamily_housing_percent_that_can_be_heated,multifamily_housing_resident_population_type
0,1407884,1076177,CitySpire,2020-12-31T00:00:00.000,New York,10019,"Fast Food Restaurant, Multifamily Housing, Off...",Multifamily Housing,Office,Restaurant,375698,369742,2063,1988,95,1,75,115.8,117.5,86773907.6,88081589.8,11.3,178364.9,1783.6,28816190.5,Not Available,2625,3.5,2625,3520.1,No,No,No,No,No,749338,Not Available,MANHATTAN,105,4,137,Midtown-Midtown South,-73.979813,40.764538,Not Available,375698,600,1.59703,20,36,340,340,0.90498,0.90498,0,0,100,100,Not Available
1,1443214,1085803,River Place One,2020-12-31T00:00:00.000,New York,10036,"Bowling Alley, Fitness Center/Health Club/Gym,...",Multifamily Housing,Parking,Bowling Alley,814289,39590,37681,2000,100,1,68,124.5,127.5,110549228.3,113225605.6,6.7,49544807.2,537657.1,20902564.5,Not Available,4292.6,4.8,4292.6,4839.3,No,No,No,No,No,887879,39590,MANHATTAN,104,3,117,Hudson Yards-Chelsea-Flatiron-Union Square,-73.999917,40.761487,No,814289,1700,2.08771,0,78,921,921,1.13105,1.13105,0,0,100,100,Not Available
2,1847014,1070014,1755 Broadway & 235 W 56th St,2020-12-31T00:00:00.000,New York,10019,"Fitness Center/Health Club/Gym, Heated Swimmin...",Multifamily Housing,Office,Parking,409506,277333,31417,1985,100,1,70,121.8,126.9,84996775.3,88498937,7.5,825446.5,9021.2,17897994.9,Not Available,3343.5,4.8,3343.5,4111.6,No,No,No,Yes,No,697639,31417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,409506,580,1.41634,0,64,482,482,1.17703,1.17703,0,0,100,100,Not Available
3,2008836,1087842,The Visionaire,2020-12-31T00:00:00.000,New York,10004,"Fitness Center/Health Club/Gym, Multifamily Ho...",Multifamily Housing,Other - Entertainment/Public Assembly,Office,401516,22500,20335,2008,100,1,15,165,165,75521750.7,75521750.7,9,34263364.5,342633.6,14123292.1,Not Available,2942.2,6.4,2942.2,2229.8,No,No,No,No,No,457771,14716,MANHAT

In [9]:
df2020.to_csv('data/multifamily_housing/2020.csv')

#### 2019

In [10]:
dataset_id = "wcm8-aq5w"

In [11]:
df_sample_2019 = pd.DataFrame(
    client.get(dataset_id, 'json', limit = 1)
)
df_sample.columns.tolist()

['property_id',
 'property_name',
 'parent_property_id',
 'parent_property_name',
 'year_ending',
 'nyc_borough_block_and_lot_bbl',
 'nyc_building_identification_number_bin',
 'address_1',
 'city',
 'postcode',
 'primary_property_type_self_selected',
 'primary_property_type_portfolio_manager_calculated',
 'national_median_reference_property_type',
 'list_of_all_property_use_types_at_property',
 'largest_property_use_type',
 'largest_property_use_type_gross_floor_area_ft',
 '_2nd_largest_property_use_type',
 '_2nd_largest_property_use_gross_floor_area_ft',
 '_3rd_largest_property_use_type',
 '_3rd_largest_property_use_type_gross_floor_area_ft',
 'year_built',
 'construction_status',
 'number_of_buildings',
 'occupancy',
 'metered_areas_energy',
 'metered_areas_water',
 'energy_star_score',
 'national_median_energy_star_score',
 'target_energy_star_score',
 'reason_s_for_no_score',
 'energy_star_certification_year_s_certified_score',
 'energy_star_certification_eligibility',
 'energy_sta

Columns to pull

In [18]:
selected_columns_2019 = [
    'property_id', 'nyc_building_identification_number_bin',
    'property_name', 'year_ending', 'city', 'postcode', 
    'list_of_all_property_use', 'largest_property_use_type', 
    '_2nd_largest_property_use', '_3rd_largest_property_use', 
    'largest_property_use_type_1', 
    '_2nd_largest_property_use_1', 
    '_3rd_largest_property_use_1', 'year_built', #!
    'occupancy', 'number_of_buildings', 'energy_star_score', 
    'source_eui_kbtu_ft', 'weather_normalized_source', 
    'source_energy_use_kbtu', 'weather_normalized_source_1', 
    'weather_normalized_site_3', # total electricity use, not per sq ft
    'natural_gas_use_kbtu', 'weather_normalized_site_2', 
    'electricity_use_grid_purchase', 
    'electricity_use_generated_2', 
    'total_ghg_emissions_metric', 
    'total_ghg_emissions_intensity', 
    'net_emissions_metric_tons', 
    'national_median_total_ghg', 
    'estimated_data_flag', 
    'estimated_data_flag_natural', 'estimated_values_energy', 
    'default_values', 'temporary_values', 'property_gfa_self_reported', 
    'property_gfa_calculated_2', 'borough', 'community_board', 
    'council_district', 'census_tract', 'nta', 'longitude', 'latitude'
]
mfh_columns = [c for c in df_sample_2019.columns if c.startswith('multifamily_housing_')]
columns = ', '.join(selected_columns_2019 + mfh_columns)
columns

'property_id, nyc_building_identification_number_bin, property_name, year_ending, city, postcode, list_of_all_property_use, largest_property_use_type, _2nd_largest_property_use, _3rd_largest_property_use, largest_property_use_type_1, _2nd_largest_property_use_1, _3rd_largest_property_use_1, year_built, occupancy, number_of_buildings, energy_star_score, source_eui_kbtu_ft, weather_normalized_source, source_energy_use_kbtu, weather_normalized_source_1, weather_normalized_site_3, natural_gas_use_kbtu, weather_normalized_site_2, electricity_use_grid_purchase, electricity_use_generated_2, total_ghg_emissions_metric, total_ghg_emissions_intensity, net_emissions_metric_tons, national_median_total_ghg, estimated_data_flag, estimated_data_flag_natural, estimated_values_energy, default_values, temporary_values, property_gfa_self_reported, property_gfa_calculated_2, borough, community_board, council_district, census_tract, nta, longitude, latitude, multifamily_housing_government, multifamily_hous

Filter to existing, metered, multifamily housing.

In [19]:
mfh1 = 'primary_property_type_self = "Multifamily Housing"'
mfh2 = 'primary_property_type = "Multifamily Housing"'
existing = 'construction_status = "Existing"'
metered = 'metered_areas_energy = "Whole Property"'
e_star = 'NOT energy_star_score = "Not Available"'
where = ' AND '.join([mfh1, mfh2, existing, metered, e_star])
where

'primary_property_type_self = "Multifamily Housing" AND primary_property_type = "Multifamily Housing" AND construction_status = "Existing" AND metered_areas_energy = "Whole Property" AND NOT energy_star_score = "Not Available"'

In [20]:
# # get all data from 2019 data set
# results = client.get(
#     dataset_id, content_type='json', 
#     limit = 100)

In [21]:
# get all data from 2019 data set
results = client.get_all(
    dataset_id, content_type='json', 
    select = columns,
    where = where)

In [22]:
# list(results)

In [23]:
# Convert to pandas DataFrame
df2019 = pd.DataFrame(results)
df2019.shape

HTTPError: 400 Client Error: Bad Request.
	Query coordinator error: query.soql.no-such-column; No such column: nyc_building_identification_number_bin; position: Map(row -> 1, column -> 23, line -> "SELECT `property_id`, `nyc_building_identification_number_bin`, `property_name`, `year_ending`, `city`, `postcode`, `list_of_all_property_use`, `largest_property_use_type`, `_2nd_largest_property_use`, `_3rd_largest_property_use`, `largest_property_use_type_1`, `_2nd_largest_property_use_1`, `_3rd_largest_property_use_1`, `year_built`, `occupancy`, `number_of_buildings`, `energy_star_score`, `source_eui_kbtu_ft`, `weather_normalized_source`, `source_energy_use_kbtu`, `weather_normalized_source_1`, `weather_normalized_site_3`, `natural_gas_use_kbtu`, `weather_normalized_site_2`, `electricity_use_grid_purchase`, `electricity_use_generated_2`, `total_ghg_emissions_metric`, `total_ghg_emissions_intensity`, `net_emissions_metric_tons`, `national_median_total_ghg`, `estimated_data_flag`, `estimate

In [ ]:
df

In [ ]:
df2019.to_csv('data/multifamily_housing/2019.csv')

#### 2018

In [ ]:
dataset_id = "4tys-3tzj"

In [ ]:
df_sample_2018 = pd.DataFrame(
    client.get(dataset_id, 'json', limit = 1)
)
df_sample_2018.columns.tolist()

['property_id',
 'property_name',
 'parent_property_id',
 'parent_property_name',
 'year_ending',
 'nyc_borough_block_and_lot',
 'nyc_building_identification',
 'address_1',
 'address_2',
 'city',
 'county',
 'postcode',
 'primary_property_type_self',
 'primary_property_type',
 'national_median_reference',
 'list_of_all_property_use',
 'largest_property_use_type',
 'largest_property_use_type_1',
 '_2nd_largest_property_use',
 '_2nd_largest_property_use_1',
 '_3rd_largest_property_use',
 '_3rd_largest_property_use_1',
 'year_built',
 'number_of_buildings',
 'occupancy',
 'metered_areas_energy',
 'metered_areas_water',
 'energy_star_score',
 'energy_star_certification',
 'energy_star_certification_1',
 'energy_star_certification_2',
 'site_eui_kbtu_ft',
 'weather_normalized_site_eui',
 'national_median_site_eui',
 'difference_from_national',
 'site_energy_use_kbtu',
 'weather_normalized_site_energy',
 'weather_normalized_site',
 'weather_normalized_site_1',
 'source_eui_kbtu_ft',
 'weath

Columns to pull

In [ ]:
selected_columns_2018 = [
    'property_id', 'nyc_building_identification', 'property_name', 
    'year_ending', 'city', 'postcode', 
    'list_of_all_property_use', 'largest_property_use_type', 
    '_2nd_largest_property_use', '_3rd_largest_property_use', 
    'largest_property_use_type_1', 
    '_2nd_largest_property_use_1', 
    '_3rd_largest_property_use_1', 'year_built', #!
    'occupancy', 'number_of_buildings', 'energy_star_score', 
    'source_eui_kbtu_ft', 'weather_normalized_source', 
    'source_energy_use_kbtu', 'weather_normalized_source_1', 
    'weather_normalized_site_3', # total electricity use, not per sq ft
    'natural_gas_use_kbtu', 'weather_normalized_site_2', 
    'electricity_use_grid_purchase', 
    'electricity_use_generated_2', 
    'total_ghg_emissions_metric', 
    # 'total_ghg_emissions_intensity', # not available
    'net_emissions_metric_tons', 
    'national_median_total_ghg', 
    'estimated_data_flag', 
    'estimated_data_flag_natural', 'estimated_values_energy', 
    'default_values', 'temporary_values', 'property_gfa_self_reported', 
    'property_gfa_calculated_2', 'borough', 'community_board', 
    'council_district', 'census_tract', 'nta', 'longitude', 'latitude'
]
mfh_columns = [c for c in df_sample_2018.columns if c.startswith('multifamily_housing_')]
columns = ', '.join(selected_columns_2018 + mfh_columns)
columns

'property_id, property_name, year_ending, city, postcode, list_of_all_property_use, largest_property_use_type, _2nd_largest_property_use, _3rd_largest_property_use, largest_property_use_type_1, _2nd_largest_property_use_1, _3rd_largest_property_use_1, year_built, occupancy, number_of_buildings, energy_star_score, source_eui_kbtu_ft, weather_normalized_source, source_energy_use_kbtu, weather_normalized_source_1, weather_normalized_site_3, natural_gas_use_kbtu, weather_normalized_site_2, electricity_use_grid_purchase, electricity_use_generated_2, total_ghg_emissions_metric, net_emissions_metric_tons, national_median_total_ghg, estimated_data_flag, estimated_data_flag_natural, estimated_values_energy, default_values, temporary_values, property_gfa_self_reported, property_gfa_calculated_2, borough, community_board, council_district, census_tract, nta, longitude, latitude, multifamily_housing_gross, multifamily_housing_government, multifamily_housing_number, multifamily_housing_number_1, mu

Filter to existing, metered, multifamily housing.

In [ ]:
mfh1 = 'primary_property_type_self = "Multifamily Housing"'
mfh2 = 'primary_property_type = "Multifamily Housing"'
# existing = 'construction_status = "Existing"'
metered = 'metered_areas_energy = "Whole Building"'
e_star = 'NOT energy_star_score = "Not Available"'
where = ' AND '.join([mfh1, mfh2, metered, e_star])
# where = ' AND '.join([mfh1, mfh2, metered, e_star])
where

'primary_property_type_self = "Multifamily Housing" AND primary_property_type = "Multifamily Housing" AND metered_areas_energy = "Whole Building" AND NOT energy_star_score = "Not Available"'

In [ ]:
# get all data from 2018 data set
results = client.get_all(
    dataset_id, content_type='json', 
    select = columns,
    where = where)

In [ ]:
# list(results)

In [ ]:
# Convert to pandas DataFrame
df2018 = pd.DataFrame(results)
df2018.shape

(15115, 57)

In [ ]:
df2018.to_csv('data/multifamily_housing/2018.csv')

#### 2017

In [ ]:
dataset_id = "4t62-jm4m"

In [ ]:
df_sample_2017 = pd.DataFrame(
    client.get(dataset_id, 'json', limit = 1)
)
df_sample_2017.columns.tolist()

['property_id',
 'year_ending',
 'generation_date',
 'account_id',
 'property_id_1',
 'property_name',
 'nyc_borough_block_and_lot',
 'nyc_building_identification',
 'address_1',
 'city',
 'postcode',
 'primary_property_type_self',
 'primary_property_type',
 'national_median_reference',
 'list_of_all_property_use',
 'largest_property_use_type',
 'largest_property_use_type_1',
 '_2nd_largest_property_use',
 '_2nd_largest_property_use_1',
 'year_built',
 'number_of_buildings',
 'occupancy',
 'metered_areas_energy',
 'metered_areas_water',
 'energy_star_score',
 'energy_star_certification',
 'energy_star_certification_1',
 'energy_star_certification_2',
 'site_eui_kbtu_ft',
 'weather_normalized_site_eui',
 'national_median_site_eui',
 'difference_from_national',
 'site_energy_use_kbtu',
 'weather_normalized_site_energy',
 'national_median_site_energy',
 'weather_normalized_site',
 'weather_normalized_site_1',
 'source_eui_kbtu_ft',
 'weather_normalized_source',
 'national_median_source_eu

Columns to pull

In [ ]:
selected_columns_2017 = [
    'property_id', 'nyc_building_identification', 'property_name', 
    'year_ending', 'city', 'postcode', 
    'list_of_all_property_use', 'largest_property_use_type', 
    '_2nd_largest_property_use', '_3rd_largest_property_use', 
    'largest_property_use_type_1', 
    '_2nd_largest_property_use_1', 
    '_3rd_largest_property_use_1', 'year_built', #!
    'occupancy', 'number_of_buildings', 'energy_star_score', 
    'source_eui_kbtu_ft', 'weather_normalized_source', 
    'source_energy_use_kbtu', 'weather_normalized_source_1', 
    'weather_normalized_site_3', # total electricity use, not per sq ft
    'natural_gas_use_kbtu', 'weather_normalized_site_2', 
    'electricity_use_grid_purchase', 
    'electricity_use_generated_2', 
    'total_ghg_emissions_metric', 
    # 'total_ghg_emissions_intensity', # not available
    'net_emissions_metric_tons', 
    'national_median_total_ghg', 
    'estimated_data_flag', 
    'estimated_data_flag_natural', 'estimated_values_energy', 
    'default_values', 'temporary_values', 'property_gfa_self_reported', 
    'property_gfa_calculated_2', 'borough', 'community_board', 
    'council_district', 'census_tract', 'nta', 'longitude', 'latitude'
]
mfh_columns = [c for c in df_sample_2017.columns if c.startswith('multifamily_housing_')]
columns = ', '.join(selected_columns_2017 + mfh_columns)
columns

'property_id, property_name, year_ending, city, postcode, list_of_all_property_use, largest_property_use_type, _2nd_largest_property_use, _3rd_largest_property_use, largest_property_use_type_1, _2nd_largest_property_use_1, _3rd_largest_property_use_1, year_built, occupancy, number_of_buildings, energy_star_score, source_eui_kbtu_ft, weather_normalized_source, source_energy_use_kbtu, weather_normalized_source_1, weather_normalized_site_3, natural_gas_use_kbtu, weather_normalized_site_2, electricity_use_grid_purchase, electricity_use_generated_2, total_ghg_emissions_metric, net_emissions_metric_tons, national_median_total_ghg, estimated_data_flag, estimated_data_flag_natural, estimated_values_energy, default_values, temporary_values, property_gfa_self_reported, property_gfa_calculated_2, borough, community_board, council_district, census_tract, nta, longitude, latitude'

In [ ]:
mfh_columns

[]

Filter to existing, metered, multifamily housing.

In [ ]:
mfh1 = 'primary_property_type_self = "Multifamily Housing"'
mfh2 = 'primary_property_type = "Multifamily Housing"'
# existing = 'construction_status = "Existing"'
metered = 'metered_areas_energy = "Whole Building"'
e_star = 'energy_star_score IS NOT NULL'
where = ' AND '.join([mfh1, mfh2, metered, e_star])
# where = ' AND '.join([mfh1, mfh2, metered, e_star])
where

'primary_property_type_self = "Multifamily Housing" AND primary_property_type = "Multifamily Housing" AND metered_areas_energy = "Whole Building" AND energy_star_score IS NOT NULL'

In [ ]:
# get all data from 2017 data set
results = client.get_all(
    dataset_id, content_type='json', 
    select = columns,
    where = where)

# results = client.get(
#     dataset_id, content_type='json', 
#     select = columns,
#     where = where,
#     limit = 50)

In [ ]:
# list(results)

In [ ]:
# Convert to pandas DataFrame
df2017 = pd.DataFrame(results)
df2017.shape

(17640, 42)

In [ ]:
df2017.to_csv('data/multifamily_housing/2017.csv')

#### 2016

In [ ]:
dataset_id = "utpj-74fz"

In [ ]:
df_sample_2016 = pd.DataFrame(
    client.get(dataset_id, 'json', limit = 1)
)
df_sample_2016.columns.tolist()

['property_id',
 'property_name',
 'parent_property_id',
 'parent_property_name',
 'year_ending',
 'nyc_borough_block_and_lot',
 'nyc_building_identification',
 'address_1',
 'address_2',
 'city',
 'county',
 'postcode',
 'primary_property_type_self',
 'primary_property_type',
 'national_median_reference',
 'list_of_all_property_use',
 'largest_property_use_type',
 'largest_property_use_type_1',
 '_2nd_largest_property_use',
 '_2nd_largest_property_use_1',
 '_3rd_largest_property_use',
 '_3rd_largest_property_use_1',
 'year_built',
 'number_of_buildings',
 'occupancy',
 'metered_areas_energy',
 'metered_areas_water',
 'energy_star_score',
 'national_median_energy_star',
 'energy_star_certification',
 'energy_star_certification_1',
 'energy_star_certification_2',
 'site_eui_kbtu_ft',
 'weather_normalized_site_eui',
 'national_median_site_eui',
 'difference_from_national',
 'site_energy_use_kbtu',
 'weather_normalized_site_energy',
 'source_energy_use_kbtu',
 'national_median_site_energy

Columns to pull

In [ ]:
selected_columns_2016 = [
    'property_id', 'nyc_building_identification', 'property_name', 
    'year_ending', 'city', 'postcode', 
    'list_of_all_property_use', 'largest_property_use_type', 
    '_2nd_largest_property_use', '_3rd_largest_property_use', 
    'largest_property_use_type_1', 
    '_2nd_largest_property_use_1', 
    '_3rd_largest_property_use_1', 'year_built', #!
    'occupancy', 'number_of_buildings', 'energy_star_score', 
    'source_eui_kbtu_ft', 'weather_normalized_source', 
    'source_energy_use_kbtu', 'weather_normalized_source_1', 
    'weather_normalized_site_3', # total electricity use, not per sq ft
    'natural_gas_use_kbtu', 'weather_normalized_site_2', 
    'electricity_use_grid_purchase', 
    'electricity_use_generated_2', 
    'total_ghg_emissions_metric', 
    # 'total_ghg_emissions_intensity', # not available
    'net_emissions_metric_tons', 
    'national_median_total_ghg', 
    'estimated_data_flag', 
    'estimated_data_flag_natural', 'estimated_values_energy', 
    'default_values', 'temporary_values', 'property_gfa_self_reported', 
    'property_gfa_calculated_2', 'borough', 'community_board', 
    'council_district', 'census_tract', 'nta', 'longitude', 'latitude'
]
mfh_columns = [c for c in df_sample_2016.columns if c.startswith('multifamily_housing_')]
columns = ', '.join(selected_columns_2016 + mfh_columns)
columns

'property_id, property_name, year_ending, city, postcode, list_of_all_property_use, largest_property_use_type, _2nd_largest_property_use, _3rd_largest_property_use, largest_property_use_type_1, _2nd_largest_property_use_1, _3rd_largest_property_use_1, year_built, occupancy, number_of_buildings, energy_star_score, source_eui_kbtu_ft, weather_normalized_source, source_energy_use_kbtu, weather_normalized_source_1, weather_normalized_site_3, natural_gas_use_kbtu, weather_normalized_site_2, electricity_use_grid_purchase, electricity_use_generated_2, total_ghg_emissions_metric, net_emissions_metric_tons, national_median_total_ghg, estimated_data_flag, estimated_data_flag_natural, estimated_values_energy, default_values, temporary_values, property_gfa_self_reported, property_gfa_calculated_2, borough, community_board, council_district, census_tract, nta, longitude, latitude, multifamily_housing_gross, multifamily_housing_government, multifamily_housing_number, multifamily_housing_number_1, mu

Filter to existing, metered, multifamily housing.

In [ ]:
mfh1 = 'primary_property_type_self = "Multifamily Housing"'
mfh2 = 'primary_property_type = "Multifamily Housing"'
# existing = 'construction_status = "Existing"'
metered = 'metered_areas_energy = "Whole Building"'
e_star = 'energy_star_score IS NOT NULL'
where = ' AND '.join([mfh1, mfh2, metered, e_star])
# where = ' AND '.join([mfh1, mfh2, metered, e_star])
where

'primary_property_type_self = "Multifamily Housing" AND primary_property_type = "Multifamily Housing" AND metered_areas_energy = "Whole Building" AND energy_star_score IS NOT NULL'

In [ ]:
# get all data from 2016 data set
results = client.get_all(
    dataset_id, content_type='json', 
    select = columns,
    where = where)

# results = client.get(
#     dataset_id, content_type='json', 
#     select = columns,
#     where = where,
#     limit = 50)

In [ ]:
# list(results)

In [ ]:
# Convert to pandas DataFrame
df2016 = pd.DataFrame(results)
df2016.shape

(10113, 57)

In [ ]:
df2016.to_csv('data/multifamily_housing/2016.csv')

#### 2015

In [ ]:
dataset_id = "77q4-nkfh"

In [ ]:
df_sample_2015 = pd.DataFrame(
    client.get(dataset_id, 'json', limit = 1)
)
df_sample_2015.columns.tolist()

['property_id',
 'property_name',
 'parent_property_id',
 'parent_property_name',
 'year_ending',
 'nyc_borough_block_and_lot',
 'nyc_building_identification',
 'address_1',
 'address_2',
 'city',
 'county',
 'postcode',
 'primary_property_type_self',
 'primary_property_type_epa',
 'national_median_reference',
 'list_of_all_property_use',
 'largest_property_use_type',
 'largest_property_use_type_1',
 '_2nd_largest_property_use',
 '_2nd_largest_property_use_1',
 '_3rd_largest_property_use',
 '_3rd_largest_property_use_1',
 'year_built',
 'number_of_buildings',
 'occupancy',
 'metered_areas_energy',
 'metered_areas_water',
 'energy_star_score',
 'national_median_energy_star',
 'energy_star_certification',
 'energy_star_certification_1',
 'energy_star_certification_2',
 'site_eui_kbtu_ft',
 'weather_normalized_site_eui',
 'national_median_site_eui',
 'difference_from_national',
 'site_energy_use_kbtu',
 'weather_normalized_site_energy',
 'national_median_site_energy',
 'weather_normalized

Columns to pull

In [ ]:
selected_columns_2015 = [
    'property_id', 'nyc_building_identification', 'property_name', 
    'year_ending', 'city', 'postcode', 
    'list_of_all_property_use', 'largest_property_use_type', 
    '_2nd_largest_property_use', '_3rd_largest_property_use', 
    'largest_property_use_type_1', 
    '_2nd_largest_property_use_1', 
    '_3rd_largest_property_use_1', 'year_built', #!
    'occupancy', 'number_of_buildings', 'energy_star_score', 
    'source_eui_kbtu_ft', 'weather_normalized_source', 
    'source_energy_use_kbtu', 'weather_normalized_source_1', 
    'weather_normalized_site_3', # total electricity use, not per sq ft
    'natural_gas_use_kbtu', 'weather_normalized_site_2', 
    'electricity_use_grid_purchase', 
    'electricity_use_generated_2', 
    'total_ghg_emissions_metric', 
    # 'total_ghg_emissions_intensity', # not available
    'net_emissions_metric_tons', 
    'national_median_total_ghg', 
    'estimated_data_flag', 
    'estimated_data_flag_natural', 'estimated_values_energy', 
    'default_values', 'temporary_values', 'property_gfa_self_reported', 
    'property_gfa_epa_calculated_2', 'borough', 'community_board', 
    'council_district', 'census_tract', 'nta', 'longitude', 'latitude'
]
mfh_columns = [c for c in df_sample_2015.columns if c.startswith('multifamily_housing_')]
columns = ', '.join(selected_columns_2015 + mfh_columns)
columns

'property_id, property_name, year_ending, city, postcode, list_of_all_property_use, largest_property_use_type, _2nd_largest_property_use, _3rd_largest_property_use, largest_property_use_type_1, _2nd_largest_property_use_1, _3rd_largest_property_use_1, year_built, occupancy, number_of_buildings, energy_star_score, source_eui_kbtu_ft, weather_normalized_source, source_energy_use_kbtu, weather_normalized_source_1, weather_normalized_site_3, natural_gas_use_kbtu, weather_normalized_site_2, electricity_use_grid_purchase, electricity_use_generated_2, total_ghg_emissions_metric, net_emissions_metric_tons, national_median_total_ghg, estimated_data_flag, estimated_data_flag_natural, estimated_values_energy, default_values, temporary_values, property_gfa_self_reported, property_gfa_epa_calculated_2, borough, community_board, council_district, census_tract, nta, longitude, latitude, multifamily_housing_gross, multifamily_housing_government, multifamily_housing_number, multifamily_housing_number_1

Filter to existing, metered, multifamily housing.

In [ ]:
mfh1 = 'primary_property_type_self = "Multifamily Housing"'
mfh2 = 'primary_property_type_epa = "Multifamily Housing"'
# existing = 'construction_status = "Existing"'
metered = 'metered_areas_energy = "Whole Building"'
e_star = 'energy_star_score IS NOT NULL'
where = ' AND '.join([mfh1, mfh2, metered, e_star])
# where = ' AND '.join([mfh1, mfh2, metered, e_star])
where

'primary_property_type_self = "Multifamily Housing" AND primary_property_type_epa = "Multifamily Housing" AND metered_areas_energy = "Whole Building" AND energy_star_score IS NOT NULL'

In [ ]:
# get all data from 2015 data set
results = client.get_all(
    dataset_id, content_type='json', 
    select = columns,
    where = where)

# results = client.get(
#     dataset_id, content_type='json', 
#     select = columns,
#     where = where,
#     limit = 50)

In [ ]:
# list(results)

In [ ]:
# Convert to pandas DataFrame
df2015 = pd.DataFrame(results)
df2015.shape

(9876, 57)

In [ ]:
df2015.to_csv('data/multifamily_housing/2015.csv')

#### 2014 and earlier

Data is available from 2010 - 2014, but some of the columns I want are unavailable so I'll leave them out.

In [ ]:
dataset_id = "nbun-wekj"

In [ ]:
df_sample_2014 = pd.DataFrame(
    client.get(dataset_id, 'json', limit = 1)
)
df_sample_2014.columns.tolist()

['property_id',
 'property_name',
 'parent_property_id',
 'parent_property_name',
 'year_ending',
 'nyc_building_identification',
 'nyc_borough_block_and_lot',
 'address_1',
 'address_2',
 'city',
 'county',
 'postcode',
 'primary_property_type_self',
 'primary_property_type_epa',
 'property_floor_area_building',
 'national_median_reference',
 'year_built',
 'number_of_buildings',
 'occupancy',
 'metered_areas_energy',
 'energy_star_score',
 'national_median_energy_star',
 'energy_star_certification',
 'energy_star_certification_1',
 'energy_star_certification_2',
 'energy_star_certification_3',
 'energy_star_certification_4',
 'energy_star_certification_5',
 'site_eui_kbtu_ft',
 'weather_normalized_site_eui',
 'national_median_site_eui',
 'difference_from_national',
 'site_energy_use_kbtu',
 'weather_normalized_site_energy',
 'national_median_site_energy',
 'weather_normalized_site',
 'weather_normalized_site_1',
 'source_eui_kbtu_ft',
 'weather_normalized_source',
 'national_median_s

Columns to pull

In [ ]:
selected_columns_2014 = [
    'property_id', 'property_name', 'year_ending', 'city', 'postcode', 
    'list_of_all_property_use', 'largest_property_use_type', 
    '_2nd_largest_property_use', '_3rd_largest_property_use', 
    'largest_property_use_type_1', 
    '_2nd_largest_property_use_1', 
    '_3rd_largest_property_use_1', 'year_built', #!
    'occupancy', 'number_of_buildings', 'energy_star_score', 
    'source_eui_kbtu_ft', 'weather_normalized_source', 
    'source_energy_use_kbtu', 'weather_normalized_source_1', 
    'weather_normalized_site_3', # total electricity use, not per sq ft
    'natural_gas_use_kbtu', 'weather_normalized_site_2', 
    'electricity_use_grid_purchase', 
    'electricity_use_generated_2', 
    'total_ghg_emissions_metric', 
    # 'total_ghg_emissions_intensity', # not available
    'net_emissions_metric_tons', 
    'national_median_total_ghg', 
    'estimated_data_flag', 
    'estimated_data_flag_natural', 'estimated_values_energy', 
    'default_values', 'temporary_values', 'property_gfa_self_reported', 
    'property_gfa_epa_calculated_2', 'borough', 'community_board', 
    'council_district', 'census_tract', 'nta', 'longitude', 'latitude'
]
mfh_columns = [c for c in df_sample_2014.columns if c.startswith('multifamily_housing_')]
columns = ', '.join(selected_columns_2014 + mfh_columns)
columns

'property_id, property_name, year_ending, city, postcode, list_of_all_property_use, largest_property_use_type, _2nd_largest_property_use, _3rd_largest_property_use, largest_property_use_type_1, _2nd_largest_property_use_1, _3rd_largest_property_use_1, year_built, occupancy, number_of_buildings, energy_star_score, source_eui_kbtu_ft, weather_normalized_source, source_energy_use_kbtu, weather_normalized_source_1, weather_normalized_site_3, natural_gas_use_kbtu, weather_normalized_site_2, electricity_use_grid_purchase, electricity_use_generated_2, total_ghg_emissions_metric, net_emissions_metric_tons, national_median_total_ghg, estimated_data_flag, estimated_data_flag_natural, estimated_values_energy, default_values, temporary_values, property_gfa_self_reported, property_gfa_epa_calculated_2, borough, community_board, council_district, census_tract, nta, longitude, latitude, multifamily_housing_government, multifamily_housing_gross, multifamily_housing_number, multifamily_housing_number_1

Filter to existing, metered, multifamily housing.

In [ ]:
mfh1 = 'primary_property_type_self = "Multifamily Housing"'
mfh2 = 'primary_property_type_epa = "Multifamily Housing"'
# existing = 'construction_status = "Existing"'
metered = 'metered_areas_energy = "Whole Building"'
e_star = 'energy_star_score IS NOT NULL'
where = ' AND '.join([mfh1, mfh2, metered, e_star])
# where = ' AND '.join([mfh1, mfh2, metered, e_star])
where

'primary_property_type_self = "Multifamily Housing" AND primary_property_type_epa = "Multifamily Housing" AND metered_areas_energy = "Whole Building" AND energy_star_score IS NOT NULL'

In [ ]:
# get all data from 2014 data set
results = client.get_all(
    dataset_id, content_type='json', 
    select = columns,
    where = where)

# results = client.get(
#     dataset_id, content_type='json', 
#     select = columns,
#     where = where,
#     limit = 50)

In [ ]:
# list(results)

In [ ]:
# Convert to pandas DataFrame
df2014 = pd.DataFrame(results)
df2014.shape

HTTPError: 400 Client Error: Bad Request.
	Query coordinator error: query.soql.no-such-column; No such column: list_of_all_property_use; position: Map(row -> 1, column -> 75, line -> "SELECT `property_id`, `property_name`, `year_ending`, `city`, `postcode`, `list_of_all_property_use`, `largest_property_use_type`, `_2nd_largest_property_use`, `_3rd_largest_property_use`, `largest_property_use_type_1`, `_2nd_largest_property_use_1`, `_3rd_largest_property_use_1`, `year_built`, `occupancy`, `number_of_buildings`, `energy_star_score`, `source_eui_kbtu_ft`, `weather_normalized_source`, `source_energy_use_kbtu`, `weather_normalized_source_1`, `weather_normalized_site_3`, `natural_gas_use_kbtu`, `weather_normalized_site_2`, `electricity_use_grid_purchase`, `electricity_use_generated_2`, `total_ghg_emissions_metric`, `net_emissions_metric_tons`, `national_median_total_ghg`, `estimated_data_flag`, `estimated_data_flag_natural`, `estimated_values_energy`, `default_values`, `temporary_values`, `p